## Bibliotecas

In [1]:
import pandas as pd                                     # Tratamiento de datos
from sklearn.model_selection import StratifiedKFold     # Clases desequelibradas
from sklearn.metrics import precision_score, f1_score, accuracy_score, recall_score
import statistics                                       # Estadisticas



## Leer dataset

In [2]:
dataset = pd.read_csv('Social_Network_Ads.csv')
dataset.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19.0,19000.0,0
1,15810944,Male,35.0,20000.0,0
2,15668575,Female,26.0,43000.0,0
3,15603246,Female,27.0,57000.0,0
4,15804002,Male,19.0,76000.0,0


## Separar rasgos y clases 

In [3]:
x = dataset.iloc[:, [2, 3]].values
y = dataset.iloc[:, 4].values

## Preprocesamiento

## Modelo a evaluar

In [4]:
from sklearn.naive_bayes import GaussianNB

## Metodos de validación

In [5]:
# --> Listas de métricas
lista_exactitud = []
lista_precision = []
lista_f1score = []
lista_sensibilidad = []

In [30]:
# --> Objeto para validacion
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

# --> Iteraciones
for entrenamiento, prueba in skf.split(x, y):
    # --> Separar datos en entrenamiento y prueba
    x_entrenamiento, x_prueba = x[entrenamiento], x[prueba]
    y_entrenamiento, y_prueba = y[entrenamiento], y[prueba]

    # --> Clasificador de bayes
    y_predicha = GaussianNB().fit(x_entrenamiento, y_entrenamiento).predict(x_prueba)

    # --> Metricas
    lista_exactitud.append(accuracy_score(y_prueba, y_predicha))
    lista_precision.append(precision_score(y_prueba, y_predicha, average="macro"))
    lista_f1score.append(f1_score(y_prueba, y_predicha, average="macro"))
    lista_sensibilidad.append(recall_score(y_prueba, y_predicha, average="macro"))


In [34]:
print(f"""
[+]Exactitud:\t {round(100*(statistics.mean(lista_exactitud)), 2)} %
[+]Presicion:\t {round(100*(statistics.mean(lista_precision)), 2)} %
[+]F1-score:\t {round(100*(statistics.mean(lista_f1score)), 2)} %
[+]Sensibilidad: {round(100*(statistics.mean(lista_sensibilidad)), 2)} %
""")



[+]Exactitud:	 88.31 %
[+]Presicion:	 88.36 %
[+]F1-score:	 86.84 %
[+]Sensibilidad: 86.39 %



## Busqueda de mejores parametros

In [35]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0)


In [36]:
from sklearn.model_selection import GridSearchCV

# --> Parametros a evaluar
parametros = {'var_smoothing': [1e-09, 1e-08, 1e-07, 1e-06, 1e-05]}

# --> Crear objeto de busqueda
busqueda = GridSearchCV(
    estimator = GaussianNB(), 
    param_grid=parametros, 
    scoring='accuracy', 
    cv=skf                       # Entero si esta balanceado el dataset
    )

# --> Entrenar objeto de busqueda
busqueda = busqueda.fit(x_train, y_train)

# --> Resultados
print(f"[+]Mejor exactitud: {100*busqueda.best_score_} %")
print(f"[+]Mejores parametros: {busqueda.best_params_}")



[+]Mejor exactitud: 87.99999999999999 %
[+]Mejores parametros: {'var_smoothing': 1e-09}


In [40]:
classifier = GaussianNB(var_smoothing=1e-09)
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)


0.9